# **Economic Development vs. Sustainability**
# Data Discovery and Structuring - CO_2 Emissions
Katlyn Goeujon-Mackness <br>
Last Updated: 17/06/2025

## Table of Contents

**1. Introduction**
- Overview of CO₂ emissions and dataset scope  
- Objectives of the analysis  

**2. Locating Relevant Data**
- Importing and exploring emissions datasets  
- Identifying structured data sources  

**3. Data Preprocessing**
- Handling missing values and inconsistencies  
- Filtering and structuring country-based emissions data  
- Assigning ISO codes to regions  

**4. Exploratory Data Analysis**
- Trends in CO₂ emissions by country and region  
- Comparisons across income groups  
- Visualizing emissions per capita and per GDP  

**5. Regional Aggregation**
- Managing large region classifications (Africa, Europe, etc.)  
- Integrating grouped emissions data  

**6. Cleaning and Refining Data**
- Filtering out non-country entities (aviation, shipping, etc.)  
- Handling missing ISO codes and country classifications  

**7. Final Adjustments**
- Validating structured emissions datasets  
- Exporting cleaned data for further analysis  



## Introduction

Economic growth is often pursued at the cost of environmental sustainability. This study aims to analyze the balance between economic development and sustainable practices across different regions, industries, and policies.

In this **discovery and structuring phase** of the data analysis, we will locate, collect and process necessary raw data to structure it in a way that will be useful in analysis. Finally, we will export processed data in CSV format for analysis.

### Key Challenge
Achieving sustainable economic growth requires balancing financial prosperity with environmental and social responsibility. Identifying actionable patterns in historical data can inform policymakers, businesses, and environmental advocates.

### Data of Interest
- GDP growth rate compared to carbon emissions per capita.
- Percentage of renewable energy adoption.
- Employment trends in green industries.
- Improvement in environmental quality indicators (air quality, water safety).
- Sustainability index scores vs. economic performance.

### Locating Relevant Data
- **World Bank**: Economic indicators.
    * [GDP per capita growth (annual %)](https://data.worldbank.org/indicator/NY.GDP.PCAP.KD.ZG)
    * [GDP per capita (constant 2015 US$)](https://data.worldbank.org/indicator/NY.GDP.PCAP.KD)
- **Our World in Data**: Environmental indicators
    * [CO2 and Greenhouse Gas Emissions](https://github.com/owid/co2-data)
- **United Nations SDGs Database**: Sustainable development statistics.
- **OECD**: Policy effectiveness on sustainability.
- **NASA Earth Observations**: Environmental impact metrics.
- **National Employment Data**: Job growth in sustainable sectors.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Prevent truncating columns and rows
pd.set_option("display.max_rows", None) 
pd.set_option("display.max_columns", None) 

In [5]:
# OWIS complete dataset
co2_data = pd.read_csv("../data/raw/emissions/owid-co2-data_raw.csv")

# gdp_groups_regions_structured to extract regional and income groups from the OWIS dataset
groups_regions = pd.read_csv("../data/in_process/1a_gdp_region_groups_structured.csv")

# access gdp_percent_structured from World Bank for the list of countries to compare to OWIS
countries = pd.read_csv("../data/in_process/1a_gdp_countries_structured.csv")

In [6]:
co2_data.head()

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,co2_including_luc,co2_including_luc_growth_abs,co2_including_luc_growth_prct,co2_including_luc_per_capita,co2_including_luc_per_gdp,co2_including_luc_per_unit_energy,co2_per_capita,co2_per_gdp,co2_per_unit_energy,coal_co2,coal_co2_per_capita,consumption_co2,consumption_co2_per_capita,consumption_co2_per_gdp,cumulative_cement_co2,cumulative_co2,cumulative_co2_including_luc,cumulative_coal_co2,cumulative_flaring_co2,cumulative_gas_co2,cumulative_luc_co2,cumulative_oil_co2,cumulative_other_co2,energy_per_capita,energy_per_gdp,flaring_co2,flaring_co2_per_capita,gas_co2,gas_co2_per_capita,ghg_excluding_lucf_per_capita,ghg_per_capita,land_use_change_co2,land_use_change_co2_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,oil_co2,oil_co2_per_capita,other_co2_per_capita,other_industry_co2,primary_energy_consumption,share_global_cement_co2,share_global_co2,share_global_co2_including_luc,share_global_coal_co2,share_global_cumulative_cement_co2,share_global_cumulative_co2,share_global_cumulative_co2_including_luc,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_luc_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,share_global_flaring_co2,share_global_gas_co2,share_global_luc_co2,share_global_oil_co2,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1750,AFG,2802560.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1751,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1752,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1753,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1754,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
groups_regions.head()

,Region Group Name,Country Code
0,Africa Eastern and Southern,AFE
1,Africa Western and Central,AFW
2,Arab World,ARB
3,Central Europe and the Baltics,CEB
4,Caribbean small states,CSS


In [8]:
countries.head()

,Country Name,Country Code
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND


### Select Data of Interest
Select the most relevant columns for the current analysis.
**Economic Performance Data**
* gdp (2011 inflation adjusted) - _Validate trends consistency with World Bank GDP_ data

**Emissions & Growth**
* co2 (annual CO_2 emissions per country)
* co2_per_capita (CO_2 emissions relative to population)
* co2_per_gdp (measures annual CO_2 emissions intensity according to GDP)

**Sustainability/Energy Effieciency**
* co2_per_unit_energy (measure carbon emissions per unit of energy used)
* energy_per_gdp (primary energy consumption per GDP - energy sustainability metric)

In [9]:
# Select relevant columns
co2_selected = co2_data[['country', 'iso_code', 'year', 'population', 'gdp', 'co2', 'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'energy_per_gdp']].copy()
co2_selected.head()

,country,iso_code,year,population,gdp,co2,co2_per_capita,co2_per_gdp,co2_per_unit_energy,energy_per_gdp
0,Afghanistan,AFG,1750,2802560.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1751,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1752,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1753,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1754,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
co2_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50191 entries, 0 to 50190
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   country              50191 non-null  object 
 1   iso_code             42262 non-null  object 
 2   year                 50191 non-null  int64  
 3   population           41019 non-null  float64
 4   gdp                  15251 non-null  float64
 5   co2                  29137 non-null  float64
 6   co2_per_capita       26182 non-null  float64
 7   co2_per_gdp          17528 non-null  float64
 8   co2_per_unit_energy  10350 non-null  float64
 9   energy_per_gdp       7696 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 3.8+ MB


In [11]:
co2_selected.describe()

,year,population,gdp,co2,co2_per_capita,co2_per_gdp,co2_per_unit_energy,energy_per_gdp
count,50191.000000,4.101900e+04,1.525100e+04,29137.000000,26182.000000,17528.000000,10350.000000,7696.000000
mean,1919.883067,5.686141e+07,3.300495e+11,415.698178,3.815391,0.397621,0.239634,1.768091
std,65.627296,3.199905e+08,3.086383e+12,1945.843973,14.383452,0.753783,0.257531,1.717964
min,1750.000000,2.150000e+02,4.998000e+07,0.000000,0.000000,0.000000,0.000000,0.078000
25%,1875.000000,3.273130e+05,7.874038e+09,0.374000,0.169000,0.131000,0.175000,0.855750
50%,1924.000000,2.289522e+06,2.743861e+10,4.990000,1.013000,0.263500,0.216000,1.294000
75%,1974.000000,9.862459e+06,1.212627e+11,53.273000,4.296750,0.508000,0.256000,2.139250
max,2023.000000,8.091735e+09,1.301126e+14,37791.570000,782.682000,82.576000,10.686000,25.253000


In [12]:
co2_selected.shape

(50191, 10)

### Standardize Countries, Regions and Groupings
Like the GDP data, the OWIS dataset includes Income and Regional groups. We will extract this subset and combine it with gdp_groups_regions from the World Bank dataset with the following steps:
1. Align consistent country codes where available
2. Standardize country and region names across datasets
3. Handle regional aggregates separately

In [13]:
# Select unique country names and their ISO codes
unique_countries_co2 = co2_selected[["country", "iso_code"]].drop_duplicates()

In [14]:
unique_countries_co2.head(10)

,country,iso_code
0,Afghanistan,AFG
274,Africa,NaN
548,Africa (GCP),NaN
722,Albania,ALB
896,Algeria,DZA
1070,Andorra,AND
1344,Angola,AGO
1518,Anguilla,AIA
1692,Antarctica,ATA
1966,Antigua and Barbuda,ATG


In [15]:
unique_countries_co2.count()

country     255
iso_code    218
dtype: int64

#### Align Consistent Country Codes Where Available

In [16]:
# Filter out countries that already have an appropriate country code 
#    according to the World Bank list in countries1.csv
# Use anti-join method
unmatched = unique_countries_co2.merge(
    countries,
    left_on=["country", "iso_code"],
    right_on=["Country Name", "Country Code"],
    how="left",
    indicator=True
).query('_merge == "left_only"').drop(columns=['_merge'])

In [17]:
unmatched = unmatched[["country", "iso_code"]]
unmatched.head(10)

,country,iso_code
1,Africa,NaN
2,Africa (GCP),NaN
7,Anguilla,AIA
8,Antarctica,ATA
13,Asia,NaN
14,Asia (GCP),NaN
15,Asia (excl. China and India),NaN
19,Bahamas,BHS
30,Bonaire Sint Eustatius and Saba,BES
35,Brunei,BRN


In [18]:
unmatched.count()

country     76
iso_code    39
dtype: int64

In [19]:
# Next, filter out any matching income/regional group that has an associated country code
unmatched2 = unmatched.merge(
    groups_regions,
    left_on="iso_code",
    right_on="Country Code",
    how="left",
    indicator=True
).query('_merge == "left_only"').drop(columns=['_merge'])


In [20]:
unmatched2 = unmatched2[["country", "iso_code"]]
unmatched2.head(10)

,country,iso_code
0,Africa,NaN
1,Africa (GCP),NaN
2,Anguilla,AIA
3,Antarctica,ATA
4,Asia,NaN
5,Asia (GCP),NaN
6,Asia (excl. China and India),NaN
7,Bahamas,BHS
8,Bonaire Sint Eustatius and Saba,BES
9,Brunei,BRN


In [21]:
unmatched2.count()

country     76
iso_code    39
dtype: int64

In [22]:
# Find matches in by country code where the names may differ
matched_regions = unmatched2.merge(
    countries,
    left_on="iso_code",
    right_on="Country Code",
    how="inner"
)
matched_regions

,country,iso_code,Country Name,Country Code
0,Bahamas,BHS,"Bahamas, The",BHS
1,Brunei,BRN,Brunei Darussalam,BRN
2,Cape Verde,CPV,Cabo Verde,CPV
3,Congo,COG,"Congo, Rep.",COG
4,Democratic Republic of Congo,COD,"Congo, Dem. Rep.",COD
5,East Timor,TLS,Timor-Leste,TLS
6,Egypt,EGY,"Egypt, Arab Rep.",EGY
7,Gambia,GMB,"Gambia, The",GMB
8,Hong Kong,HKG,"Hong Kong SAR, China",HKG
9,Iran,IRN,"Iran, Islamic Rep.",IRN


#### Update co2_data to reflect the World Data country names

In [23]:
# Set mapping to come from countries dataframe
country_mapping = countries.set_index("Country Code")["Country Name"]

# Update country name in co2_data using iso_code
co2_selected["country"] = co2_selected["iso_code"].map(country_mapping).fillna(co2_selected["country"])

In [24]:
# Show the remaining regions/income groups without country codes
unmatched3 = co2_selected[co2_selected['iso_code'].isnull()][['country', 'iso_code']].drop_duplicates().head(20)
unmatched3

,country,iso_code
274,Africa,NaN
548,Africa (GCP),NaN
2666,Asia,NaN
2940,Asia (GCP),NaN
3114,Asia (excl. China and India),NaN
9184,Central America (GCP),NaN
14919,Europe,NaN
15193,Europe (GCP),NaN
15367,Europe (excl. EU-27),NaN
15641,Europe (excl. EU-28),NaN


In [25]:
# Regional GCP data appears to be redundant and will be removed from the DataFrame
co2_selected = co2_selected[~co2_selected["country"].str.contains("\(GCP\)", case=False, na=False)]
#   In addition to other categories not relevant to the current analysis

# Show the remaining regions/income groups without country codes
unmatched4 = co2_selected[co2_selected['iso_code'].isnull()][['country', 'iso_code']].drop_duplicates()
unmatched4


<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
/tmp/ipykernel_2866/3499219293.py:2: SyntaxWarning: invalid escape sequence '\('
  co2_selected = co2_selected[~co2_selected["country"].str.contains("\(GCP\)", case=False, na=False)]


,country,iso_code
274,Africa,NaN
2666,Asia,NaN
3114,Asia (excl. China and India),NaN
14919,Europe,NaN
15367,Europe (excl. EU-27),NaN
15641,Europe (excl. EU-28),NaN
15915,European Union (27),NaN
16189,European Union (28),NaN
20297,High-income countries,NaN
21833,International aviation,NaN


In [26]:
# Match income-related categories
filtered_income_countries = co2_selected[co2_selected["country"].str.contains("income", case=False, na=False)][["country", "iso_code"]].drop_duplicates()
filtered_income_countries2 = groups_regions[groups_regions["Region Group Name"].str.contains("income", case=False, na=False)][["Region Group Name", "Country Code"]].drop_duplicates()

print(filtered_income_countries)
print(filtered_income_countries2)


                             country iso_code
20297          High-income countries      NaN
26736           Low-income countries      NaN
27010  Lower-middle-income countries      NaN
47800  Upper-middle-income countries      NaN
                                    Region Group Name Country Code
5         East Asia & Pacific (excluding high income)          EAP
8       Europe & Central Asia (excluding high income)          ECA
13                                        High income          HIC
20  Latin America & Caribbean (excluding high income)          LAC
23                                         Low income          LIC
24                                Lower middle income          LMC
25                                Low & middle income          LMY
28                                      Middle income          MIC
29  Middle East & North Africa (excluding high inc...          MNA
37         Sub-Saharan Africa (excluding high income)          SSA
46                               

In [27]:
# Define mapping for country name and ISO codes
income_mapping = {
    "High-income countries": ("High income", "HIC"),
    "Low-income countries": ("Low income", "LIC"),
    "Lower-middle-income countries": ("Lower middle income", "LMC"),
    "Upper-middle-income countries": ("Upper middle income", "UMC")
}

# Apply transformations
co2_selected["country"] = co2_selected["country"].str.strip()
mask = co2_selected["country"].isin(income_mapping.keys())
co2_selected.loc[mask, ["country", "iso_code"]] = co2_selected["country"].map(income_mapping).apply(pd.Series)

# Show the remaining regions/income groups without country codes
unmatched5 = co2_selected[co2_selected['iso_code'].isnull()][['country', 'iso_code']].drop_duplicates()
unmatched5

,country,iso_code
274,Africa,NaN
2666,Asia,NaN
3114,Asia (excl. China and India),NaN
14919,Europe,NaN
15367,Europe (excl. EU-27),NaN
15641,Europe (excl. EU-28),NaN
15915,European Union (27),NaN
16189,European Union (28),NaN
20297,NaN,NaN
21833,International aviation,NaN


In [28]:
# Define country codes to remaining regions
region_iso_mapping = {
    "Africa": "AFR",
    "Asia": "ASI",
    "Asia (excl. China and India)": "ASI-XCNIN",
    "Europe": "EUR",
    "Europe (excl. EU-27)": "EUR-X27",
    "Europe (excl. EU-28)": "EUR-X28",
    "European Union (27)": "EU-27",
    "European Union (28)": "EU-28",
    "North America": "NAM",
    "North America (excl. USA)": "NAM-XUSA",
    "Oceania": "OCE",
    "South America": "SAM",
    "World": "WLD"
}

# Integrate ISO codes into co2_selected dataframe
co2_selected.loc[co2_selected["iso_code"].isna(), "iso_code"] = co2_selected["country"].map(region_iso_mapping)

# Convert mapping to DataFrame
region_groups_df = pd.DataFrame(list(region_iso_mapping.items()), columns=["Region Group Name", "Country Code"])

# Append to existing `regions_groups`
groups_regions = pd.concat([groups_regions, region_groups_df], ignore_index=True)

print(groups_regions)

                                    Region Group Name Country Code
0                         Africa Eastern and Southern          AFE
1                          Africa Western and Central          AFW
2                                          Arab World          ARB
3                      Central Europe and the Baltics          CEB
4                              Caribbean small states          CSS
5         East Asia & Pacific (excluding high income)          EAP
6                          Early-demographic dividend          EAR
7                                 East Asia & Pacific          EAS
8       Europe & Central Asia (excluding high income)          ECA
9                               Europe & Central Asia          ECS
10                                          Euro area          EMU
11                                     European Union          EUU
12           Fragile and conflict affected situations          FCS
13                                        High income         

In [29]:
# Show the remaining regions/income groups without country codes
unmatched6 = co2_selected[co2_selected['iso_code'].isnull()][['country', 'iso_code']].drop_duplicates()
unmatched6

,country,iso_code
20297,NaN,NaN
21833,International aviation,NaN
22107,International shipping,NaN
22381,International transport,NaN
24433,Kosovo,NaN
24780,Kuwaiti Oil Fires,NaN
25514,Least developed countries (Jones et al.),NaN
34946,OECD (Jones et al.),NaN
38332,Ryukyu Islands,NaN


In [30]:
# Remove the remaining list of groups
# Define the list of countries to remove
remove_list = [
    "International aviation", "International shipping", "International transport",
    "Kuwaiti Oil Fires", "Least developed countries (Jones et al.)", "OECD (Jones et al.)"
]
# Remove matching rows from co2_selected
co2_selected = co2_selected[~co2_selected["country"].isin(remove_list)]

In [31]:
# Verify complete regions/income groupings 
groups_regions.head(5)

,Region Group Name,Country Code
0,Africa Eastern and Southern,AFE
1,Africa Western and Central,AFW
2,Arab World,ARB
3,Central Europe and the Baltics,CEB
4,Caribbean small states,CSS


---
## Export data
### Export revised region/income groupings

In [ ]:
# Comment out to avoid duplicate exports
# groups_regions.to_csv("../data/in_process/1b_co2_region_groups_structured.csv")

### Export CO_2 Emissions Selected Dataset

In [ ]:
# Comment out to avoid duplicate exports
# co2_selected.to_csv("../data/in_process/1b_co2_structured.csv")